In [1]:
import pandas as pd
import pickle

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv', encoding='ISO-8859-1')

In [6]:
!pip install geopy

     ------------------------------------ 125.4/125.4 kB 184.3 kB/s eta 0:00:00
     ---------------------------------------- 40.3/40.3 kB 1.9 MB/s eta 0:00:00


In [7]:
from geopy.distance import geodesic
import requests

def get_coordinates(pincode):
    url = f'https://nominatim.openstreetmap.org/search?q={pincode}&format=json&addressdetails=1'
    response = requests.get(url)
    data = response.json()
    if len(data) > 0:
        return data[0]['lat'], data[0]['lon']
    else:
        return '0', '0'

In [8]:
pd.set_option('display.max_columns', None)
train.isnull().sum()

CandidateID                                                                      0
Designation                                                                      0
Have you Completed your Graduation ?                                           101
Highest Educational Qualification                                              101
Total no of years Experience [before joining Piramal]                           74
Previous Industry worked with [before joining Piramal]                           0
Name of your Previous Organization / Company                                   192
How many Organization that you have worked before joining Piramal Finance ?     74
Average Incentive [per month] earned in your pervious company ?                  0
How did you come to know about the role at Piramal Finance ?                     0
Which Products you are selling in your pervious role ?                           6
What was the average ticket size handled at your end in previous role ?          0
How 

In [9]:
col8 = 'Branch Pincode'
col9 = 'Residential Pincode'
train[col8] = train[col8].fillna(train[col9])
test[col8] = test[col8].fillna(train[col9])

In [10]:
from tqdm import tqdm

In [11]:
lat_rl = []
lon_rl = []
for i in tqdm (range (745), desc="Counting...") :
    lat_r, lon_r = get_coordinates(train['Residential Pincode'][i]) 
    lat_rl.append(lat_r)
    lon_rl.append(lon_r)
len(lat_rl)

Counting...:  57%|██████████████████████████████████████▍                            | 428/745 [14:59<11:06,  2.10s/it]


SSLError: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=500013&format=json&addressdetails=1 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1007)')))

In [38]:
lat_bl = []
lon_bl = []
for i in tqdm (range (745), desc="Counting...") :
    lat_b, lon_b = get_coordinates(train['Branch Pincode'][i]) 
    lat_bl.append(lat_b)
    lon_bl.append(lon_b)

Counting...: 100%|███████████████████████████████████████████████████████████████████| 745/745 [21:41<00:00,  1.75s/it]


In [9]:
lat_rlt = []
lon_rlt = []
for i in tqdm (range (187), desc="Counting...") :
    lat_r, lon_r = get_coordinates(test['Residential Pincode'][i]) 
    lat_rlt.append(lat_r)
    lon_rlt.append(lon_r)

Counting...: 100%|███████████████████████████████████████████████████████████████████| 187/187 [05:03<00:00,  1.62s/it]


In [10]:
lat_blt = []
lon_blt = []
for i in tqdm (range (187), desc="Counting...") :
    lat_b, lon_b = get_coordinates(test['Branch Pincode'][i]) 
    lat_blt.append(lat_b)
    lon_blt.append(lon_b)

Counting...: 100%|███████████████████████████████████████████████████████████████████| 187/187 [05:31<00:00,  1.77s/it]


In [41]:
train['branch_lat'] = pd.DataFrame(lat_bl, columns=['branch_lat'])
train['branch_lon'] = pd.DataFrame(lon_bl, columns=['branch_lon'])
train['residential_lat'] = pd.DataFrame(lat_rl, columns=['residential_lat'])
train['residential_lon'] = pd.DataFrame(lon_rl, columns=['residential_lon'])

In [42]:
test['branch_lat'] = pd.DataFrame(lat_blt, columns=['branch_lat'])
test['branch_lon'] = pd.DataFrame(lon_blt, columns=['branch_lon'])
test['residential_lat'] = pd.DataFrame(lat_rlt, columns=['residential_lat'])
test['residential_lon'] = pd.DataFrame(lon_rlt, columns=['residential_lon'])

In [39]:
import pickle
with open("lat_bl.pkl", "wb") as file:
    pickle.dump(lat_bl, file)

# with open("lon_bl.pkl", "wb") as file:
#     pickle.dump(lon_bl, file)

# with open("lat_rl.pkl", "wb") as file:
#     pickle.dump(lat_rl, file)

# with open("lon_rl.pkl", "wb") as file:
#     pickle.dump(lon_rl, file)

# with open("lat_blt.pkl", "wb") as file:
#     pickle.dump(lat_blt, file)

# with open("lon_blt.pkl", "wb") as file:
#     pickle.dump(lon_blt, file)

# with open("lat_rlt.pkl", "wb") as file:
#     pickle.dump(lat_rlt, file)

# with open("lon_rlt.pkl", "wb") as file:
#     pickle.dump(lon_rlt, file)

In [40]:
with open("lat_bl.pkl", "rb") as file:
    lat_bl = pickle.load(file)

# with open("lon_bl.pkl", "rb") as file:
#     lon_bl = pickle.load(file)

# with open("lat_rl.pkl", "rb") as file:
#     lat_rl = pickle.load(file)

# with open("lon_rl.pkl", "rb") as file:
#     lon_rl = pickle.load(file)

# with open("lat_blt.pkl", "rb") as file:
#     lat_blt = pickle.load(file)

# with open("lon_blt.pkl", "rb") as file:
#     lon_blt = pickle.load(file)

# with open("lat_rlt.pkl", "rb") as file:
#     lat_rlt = pickle.load(file)

# with open("lon_rlt.pkl", "rb") as file:
#     lon_rlt = pickle.load(file)

In [43]:
def calculate_distance(lat1, lon1, lat2, lon2):
    if (pd.notna(lat1) and pd.notna(lon1) and pd.notna(lat2) and pd.notna(lon2)):
        point1 = (lat1, lon1)
        point2 = (lat2, lon2)
        return geodesic(point1, point2).km
    else:
        return None

train['distance'] = train.apply(lambda row: calculate_distance(row['branch_lat'], row['branch_lon'], row['residential_lat'], row['residential_lon']), axis=1)
test['distance'] = test.apply(lambda row: calculate_distance(row['branch_lat'], row['branch_lon'], row['residential_lat'], row['residential_lon']), axis=1)

In [47]:
train.to_csv('train_dist.csv', index=False)
test.to_csv('test_dist.csv', index=False)

In [44]:
train['education'] = train['Have you Completed your Graduation ?'] + ' ' + train['Highest Educational Qualification']
train = train.drop(['Have you Completed your Graduation ?', 'Highest Educational Qualification', 'branch_lat','branch_lon', 'residential_lat', 'residential_lon'], axis=1)
test['education'] = test['Have you Completed your Graduation ?'] + ' ' + test['Highest Educational Qualification']
test = test.drop(['Have you Completed your Graduation ?', 'Highest Educational Qualification', 'branch_lat','branch_lon', 'residential_lat', 'residential_lon'], axis=1)

KeyError: 'Have you Completed your Graduation ?'

, 'branch_lat','branch_lon', 'residential_lat', 'residential_lon'

In [15]:
with open("resume_data_new.pkl", "rb") as file:
    resume_text_new = pickle.load(file)

In [16]:
# resume_txt = pd.DataFrame(resume_text, columns=['resume_txt'])
# df = pd.concat([train, resume_txt], axis=1)

In [17]:
#optional
with open("resume_data_test_new1.pkl", "rb") as file:
    resume_text_test_new1 = pickle.load(file)

In [18]:
with open("resume_data_test_new2.pkl", "rb") as file:
    resume_text_test_new2 = pickle.load(file)

In [19]:
 resume_text_test_new =  resume_text_test_new1 +  resume_text_test_new2

In [29]:
import os
import docx2txt
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from tqdm import tqdm

# # Load the training and test data
# train_dir = 'CV train'
# test_dir = 'CV test'

# # Load the training data
# train_docs = []
# for filename in os.listdir(train_dir):
#     if filename.endswith('.docx'):
#         file_path = os.path.join(train_dir, filename)
#         text = docx2txt.process(file_path)
#         train_docs.append(text)

# # Load the test data
# test_docs = []
# for filename in os.listdir(test_dir):
#     if filename.endswith('.docx'):
#         file_path = os.path.join(test_dir, filename)
#         text = docx2txt.process(file_path)
#         test_docs.append(text)

# Convert the text to tagged documents
train_tagged = [TaggedDocument(doc.split(), [i]) for i, doc in enumerate(resume_text_new)]
test_tagged = [TaggedDocument(doc.split(), [i]) for i, doc in enumerate(resume_text_test_new)]

# Train the Doc2Vec model
model = Doc2Vec(vector_size=700, min_count=2, epochs=40)
model.build_vocab(train_tagged)
model.train(train_tagged, total_examples=model.corpus_count, epochs=model.epochs)

# Get the document vectors
train_vectors = [model.infer_vector(doc.words) for doc in tqdm(train_tagged)]
test_vectors = [model.infer_vector(doc.words) for doc in tqdm(test_tagged)]

100%|████████████████████████████████████████████████████████████████████████████████| 187/187 [00:04<00:00, 42.68it/s]


In [30]:
train['train_vector'] = train_vectors
test['train_vector'] = test_vectors

In [29]:
#test = test.rename(columns = {'test_vector': 'train_vector'})

In [30]:
#resume_test = pd.DataFrame(resume_text_test, columns = ['resume_txt'])

In [31]:
#test = pd.concat([test, resume_test], axis=1)

In [31]:
train.to_csv('train1.csv', index=False)
test.to_csv('test1.csv', index=False)

In [32]:
from autogluon.core.dataset import TabularDataset
train_data = TabularDataset('train1.csv')
train_data.head(30)
train_data.columns

Loaded data from: train1.csv | Columns = 21 / 21 | Rows = 745 -> 745


Index(['CandidateID', 'Designation',
       'Total no of years Experience [before joining Piramal]',
       'Previous Industry worked with [before joining Piramal]',
       'Name of your Previous Organization / Company',
       'How many Organization that you have worked before joining Piramal Finance ?',
       'Average Incentive [per month] earned in your pervious company ?',
       'How did you come to know about the role at Piramal Finance ?',
       'Which Products you are selling in your pervious role ?',
       'What was the average ticket size handled at your end in previous role ?',
       'How many members are there in your family ?',
       'How many are earning family members ? [Other then yourself]2',
       'How many members are dependent on you ?', 'Department', 'DOJ',
       'Location Code', 'Residential Pincode', 'Branch Pincode', 'Performance',
       'education', 'train_vector'],
      dtype='object')

In [33]:
from autogluon.tabular import TabularPredictor 
time_limit = 60
predictor = TabularPredictor(label='Performance', eval_metric='f1').fit(train_data)

No path specified. Models will be saved in: "AutogluonModels\ag-20240411_093252"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20240411_093252"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          16
Memory Avail:     

In [34]:
test_data = TabularDataset('test1.csv')
label = 'CandidateID'
y_pred = predictor.predict((test_data))
y_pred.head()

Loaded data from: test1.csv | Columns = 20 / 20 | Rows = 187 -> 187


0    1
1    1
2    0
3    0
4    0
Name: Performance, dtype: int64

In [36]:
sub=pd.DataFrame()
sub['CandidateID'],sub['Performance']=test_data['CandidateID'],y_pred
sub.to_csv('submission_6.csv', index=False)
sub

,CandidateID,Performance
0,EMP0521,1
1,EMP0613,1
2,EMP0136,0
3,EMP0351,0
4,EMP0049,0
...,...,...
182,EMP0401,1
183,EMP0408,0
184,EMP0248,0
185,EMP0148,0


In [31]:
predictor.feature_importance(train_data)

These features in provided data are not utilized by the predictor and will be ignored: ['CandidateID', 'Designation']
Computing feature importance via permutation shuffling for 16 features using 745 rows with 5 shuffle sets...
C:\Users\91939\anaconda3\lib\site-packages\autogluon\tabular\models\fastainn\tabular_nn_fastai.py:200: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  df = df.fillna(column_fills, inplace=False, downcast=False)
	58.2s	= Expected runtime (11.64s per shuffle set)
C:\Users\91939\anaconda3\lib\site-packages\autogluon\tabular\models\fastainn\tabular_nn_fastai.py:200: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to down

,importance,stddev,p_value,n,p99_high,p99_low
DOJ,0.140403,0.011801,0.000006,5,0.164702,0.116103
Which Products you are selling in your pervious role ?,0.093423,0.010847,0.000021,5,0.115756,0.071089
train_vector,0.088322,0.009222,0.000014,5,0.107310,0.069334
What was the average ticket size handled at your end in previous role ?,0.053960,0.012207,0.000294,5,0.079093,0.028826
Name of your Previous Organization / Company,0.035168,0.004783,0.000040,5,0.045017,0.025318
Average Incentive [per month] earned in your pervious company ?,0.033826,0.004877,0.000050,5,0.043867,0.023784
education,0.023624,0.005583,0.000348,5,0.035120,0.012129
Department,0.019866,0.001750,0.000007,5,0.023469,0.016262
Total no of years Experience [before joining Piramal],0.013691,0.002206,0.000078,5,0.018233,0.009150
How did you come to know about the role at Piramal Finance ?,0.012886,0.004308,0.001299,5,0.021756,0.004016


In [34]:
time_limit = 60
predictor1 = TabularPredictor(label='Performance').fit(train_data, time_limit=time_limit, presets='high_quality')

No path specified. Models will be saved in: "AutogluonModels\ag-20240410_200843"
Presets specified: ['high_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 60 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels\ag-20240410_200843/ds_sub_fit/sub_fit_ho.
Running the sub-fit in a ray process to avoid memory leakage.
Spend 22 seconds for the sub-fit(s) during dynamic stacking.
Time left for full fit of AutoGluon: 38 seconds.
Starting full fit now with num_stack_levels 